In [1]:
import geopandas as gpd
import rasterio
from rasterio import features
import pandas as pd
import numpy as np
import os
from shapely.geometry import LineString
from pyproj import CRS


os.chdir('/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/Data')

flowlines=gpd.read_file('flowlines_with_pop_density.geojson')
dem = rasterio.open('output_USGS30m.tif')

In [2]:
# Ensure both datasets are in the same CRS
dem_crs = CRS(dem.crs)

if flowlines.crs != dem_crs:
    flowlines = flowlines.to_crs(dem_crs)

def sample_elevations_along_line(line_geom, dem, num_samples=100):
    """
    Samples elevations along a line geometry from a DEM raster.
    
    Parameters:
    - line_geom: Shapely LineString geometry of the flowline.
    - dem: Opened rasterio DEM object.
    - num_samples: Number of points to sample along the line.

    Returns:
    - elevations: List of elevation values sampled from the DEM.
    """
    # Generate equally spaced points along the line
    distances = np.linspace(0, line_geom.length, num_samples)
    points = [line_geom.interpolate(distance) for distance in distances]
    
    # Get the coordinates of the points
    coords = [(point.x, point.y) for point in points]
    
    # Sample the DEM at these points
    elevations = []
    for coord in coords:
        try:
            # Sample the DEM at the coordinate
            elevation = list(dem.sample([coord]))[0][0]
            if dem.nodata is not None and elevation == dem.nodata:
                elevations.append(None)
            else:
                elevations.append(elevation)
        except Exception:
            elevations.append(None)
    
    # Filter out None or NaN values
    elevations = [e for e in elevations if e is not None and not np.isnan(e)]
    return elevations

# Prepare a list to store the results
results = []

for idx, row in flowlines.iterrows():
    line_geom = row.geometry
    
    # Sample elevations along the line
    elevations = sample_elevations_along_line(line_geom, dem)
    
    # Calculate statistics if we have valid elevations
    if elevations:
        max_elev = max(elevations)
        min_elev = min(elevations)
        avg_elev = sum(elevations) / len(elevations)
    else:
        max_elev = min_elev = avg_elev = None
    
    # Store the original index, plus your stats
    results.append({
        'flow_idx': idx,            # The original index from flowlines
        'Max_Elevation': max_elev,
        'Min_Elevation': min_elev,
        'Avg_Elevation': avg_elev
    })

# Create a DataFrame from the results
elevation_df = pd.DataFrame(results)

# Make 'flow_idx' the index
elevation_df.set_index('flow_idx', inplace=True)

# Export the results to an Excel file
# Replace 'flowline_elevations.xlsx' with your desired output file name
elevation_df.to_excel('flowline_elevations.xlsx', index=False)

print("Elevation statistics calculated and saved to 'flowline_elevations.xlsx'.")

Elevation statistics calculated and saved to 'flowline_elevations.xlsx'.


In [3]:
flowlines_merged = flowlines.join(elevation_df, how='left')

In [5]:
flowlines_merged.head()

,ACTIONDESCRIPTION,BEDDINGMATERIAL,COMPANY_NAME,CONSTRUCTDATE,Diam_in,ENDLAT,ENDLONG,ENTIRELINEREMOVED,FLOWLINEACTION,FLOWLINEID,...,STARTLAT,STARTLOCATIONID,STARTLONG,Status,TYPEOFFLUIDTRANS,average_pop_density,geometry,Max_Elevation,Min_Elevation,Avg_Elevation
0,The flowline serving the Emerson 3-29J (05-123...,Native Materials,CRESTONE PEAK RESOURCES OPERATING LLC ...,1983-11-09,2.0,40.109444,-104.909686,None,None,470446.0,...,40.105743,NaN,-104.909860,Active,Multiphase,549.8,"MULTILINESTRING ((-104.90986 40.10574, -104.90...",1536.651001,1529.409180,1533.157471
1,The flowline (12311399_FL) servicing the Emers...,Native Materials,CRESTONE PEAK RESOURCES OPERATING LLC ...,1983-12-07,2.0,40.109441,-104.909670,True,Out of Service,470445.0,...,40.112203,319521.0,-104.909862,Active,Multiphase,549.8,"MULTILINESTRING ((-104.90986 40.1122, -104.909...",1530.072388,1526.790649,1528.087646
2,None,None,PETERSON ENERGY OPERATING INC ...,2006-06-05,2.0,40.063138,-105.034086,None,None,466061.0,...,40.063823,305594.0,-105.030068,Active,Multiphase,173.4,"MULTILINESTRING ((-105.03484 40.06362, -105.03...",1542.665161,1540.902710,1542.131714
3,None,None,PETERSON ENERGY OPERATING INC ...,2007-04-08,2.0,40.063142,-105.034085,None,None,466062.0,...,40.060063,305595.0,-105.034795,Active,Multiphase,173.4,"MULTILINESTRING ((-105.03484 40.06362, -105.03...",1542.665161,1540.902710,1542.131714
4,None,None,PETERSON ENERGY OPERATING INC ...,2005-05-10,2.0,40.063132,-105.034086,None,None,466063.0,...,40.059581,332338.0,-105.029331,Active,Multiphase,173.4,"MULTILINESTRING ((-105.03484 40.06362, -105.03...",1542.665161,1540.902710,1542.131714


In [4]:
flowlines_merged.to_file("flowlines_pop_dems.geojson", driver='GeoJSON')